In [ ]:
# https://www.kaggle.com/averkij/opennmt-machine-translation-baseline/notebook#Train-&-test-split
# https://git.swisscom.com/projects/DPA/repos/plato-ai-gswtranslation/browse/training/train.py
# https://github.com/Parkchanjun/OpenNMT-Colab-Tutorial/blob/master/OpenNMT_Pytorch_Tutorial.ipynb
!git clone https://github.com/OpenNMT/OpenNMT-py
!mkdir /kaggle/working/data
!mkdir /kaggle/working/output

In [ ]:
train_count = 100000
val_count = 5000
test_count = 5000

In [ ]:
with open('/kaggle/input/open_ru.txt', mode='r', encoding='utf-8') as ru_in, open('/kaggle/working/data/sent_ru_train.txt', mode='w', encoding='utf-8') as ru_out_train, \
     open('/kaggle/working/data/sent_ru_val.txt', mode='w', encoding='utf-8') as ru_out_val, open('/kaggle/working/data/sent_ru_test.txt', mode='w', encoding='utf-8') as ru_out_test:

    for i in range(train_count):
        if i < train_count-1:
            ru_out_train.write(ru_in.readline().strip() + '\n')
        else:
            ru_out_train.write(ru_in.readline().strip())

    for i in range(val_count):
        if i < val_count-1:
            ru_out_val.write(ru_in.readline().strip() + '\n')
        else:
            ru_out_val.write(ru_in.readline().strip())

    for i in range(test_count):
        if i < test_count-1:
            ru_out_test.write(ru_in.readline().strip() + '\n')
        else:
            ru_out_test.write(ru_in.readline().strip())

In [ ]:
with open('/kaggle/input/open_zh.txt', mode='r', encoding='utf-8') as zh_in, open('/kaggle/working/data/sent_zh_train.txt', mode='w', encoding='utf-8') as zh_out_train, \
     open('/kaggle/working/data/sent_zh_val.txt', mode='w', encoding='utf-8') as zh_out_val, open('/kaggle/working/data/sent_zh_test.txt', mode='w', encoding='utf-8') as zh_out_test:

    for i in range(train_count):
        if i < train_count-1:
            zh_out_train.write(zh_in.readline().strip() + '\n')
        else:
            zh_out_train.write(zh_in.readline().strip())

    for i in range(val_count):
        if i < val_count-1:
            zh_out_val.write(zh_in.readline().strip() + '\n')
        else:
            zh_out_val.write(zh_in.readline().strip())

    for i in range(test_count):
        if i < test_count-1:
            zh_out_test.write(zh_in.readline().strip() + '\n')
        else:
            zh_out_test.write(zh_in.readline().strip())

In [ ]:
!python /kaggle/working/OpenNMT-py/tools/learn_bpe.py -i /kaggle/working/data/sent_zh_train.txt -o /kaggle/working/data/src.code -s 32000
!python /kaggle/working/OpenNMT-py/tools/learn_bpe.py -i /kaggle/working/data/sent_ru_train.txt -o /kaggle/working/data/trg.code -s 32000

In [ ]:
!python /kaggle/working/OpenNMT-py/tools/apply_bpe.py -c /kaggle/working/data/src.code -i /kaggle/working/data/sent_zh_train.txt -o /kaggle/working/data/src-train-bpe.txt
!python /kaggle/working/OpenNMT-py/tools/apply_bpe.py -c /kaggle/working/data/src.code -i /kaggle/working/data/sent_zh_val.txt -o /kaggle/working/data/src-val-bpe.txt
!python /kaggle/working/OpenNMT-py/tools/apply_bpe.py -c /kaggle/working/data/src.code -i /kaggle/working/data/sent_zh_test.txt -o /kaggle/working/data/src-test-bpe.txt
!python /kaggle/working/OpenNMT-py/tools/apply_bpe.py -c /kaggle/working/data/trg.code -i /kaggle/working/data/sent_ru_train.txt -o /kaggle/working/data/tgt-train-bpe.txt
!python /kaggle/working/OpenNMT-py/tools/apply_bpe.py -c /kaggle/working/data/trg.code -i /kaggle/working/data/sent_ru_val.txt -o /kaggle/working/data/tgt-val-bpe.txt

In [ ]:
!python /kaggle/working/OpenNMT-py/preprocess.py -train_src /kaggle/working/data/src-train-bpe.txt -train_tgt /kaggle/working/data/tgt-train-bpe.txt \
    -valid_src /kaggle/working/data/src-val-bpe.txt -valid_tgt /kaggle/working/data/tgt-val-bpe.txt -save_data /kaggle/working/data/ondatr \
    -src_vocab_size 32000 -tgt_vocab_size 32000

In [ ]:
!python /kaggle/working/OpenNMT-py/train.py -data /kaggle/working/data/ondatr -save_model /kaggle/working/data/ondatr-trans -world_size 1 \
    -gpu_rank 0 --train_steps 5000 -save_checkpoint_steps 5000 --keep_checkpoint 10


In [ ]:
!python /kaggle/working/OpenNMT-py/translate.py -model /kaggle/working/data/ondatr-trans_step_5000.pt -src /kaggle/working/data/src-test-bpe.txt \
    -output /kaggle/working/output/pred.bpe -replace_unk -verbose